In [158]:
import pandas as pd
from functools import reduce
import numpy as np

In [159]:
path = "data_files/"
column_names = ['Calendar Day', 'Stock', 'Moving Average', 'Balance Sheet', 'Cash Flow', 'Income Statement',
                'Shares Outstanding']
metrics = ["BALANCE_SHEET", "CASH_FLOW", "INCOME_STATEMENT", "SHARES_OUTSTANDING"]
stocks = ["AAPL"]
daily_metrics = ["DAILY_VOLATILITY", "MOVING_AVERAGE"]
yearly_metrics = ["BALANCE_SHEET", "CASH_FLOW", "INCOME_STATEMENT"]

In [160]:
print("hello")

hello


In [85]:
calendar_codes = {"FOMC": 1, "HalfDays": 2, "MonthEnd": 3, "MSCIrebal": 4, "OptionExpiry": 5, "QuarterEnd": 6, "RusselRebal": 7, "TripleWitch": 8}

In [86]:
def combine_datasets_by_stock(stock, daily_metrics):
    stock_dataframes = []
    for metric in daily_metrics:
        df = pd.read_csv(f"{path + stock}_{metric}.csv.gz")
        stock_dataframes.append(df)
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='time_stamp', how='outer'), stock_dataframes)
    merged_df['time_stamp'] = pd.to_datetime(merged_df['time_stamp'])
    merged_df.insert(loc=0, column='Stock', value=stock)
    return merged_df
    

In [87]:
def add_cal_dates(calendar_codes, df):
    df["CALENDAR_DAYS"] = 0
    for day in list(calendar_codes.keys()):
        calendar_df = pd.read_csv(f"{path+day}_CALDATES.csv.gz")
        for i in calendar_df['time_stamp']:
            df.loc[df['time_stamp'] == i, "CALENDAR_DAYS"] = calendar_codes[day]
    return df

In [ ]:
# def add_yearly_metrics(df, yearly_metrics, stock):
#     for metric in yearly_metrics:
#         yearly_metric_df = pd.read_csv(f"{path + stock}_{metric}.csv.gz")
#         prefix = f'{metric}: '
#         excluded_col = 'time_stamp'
#         yearly_metric_df = [f'{prefix}{col}' if col != excluded_col else col for col in yearly_metric_df.columns]
#         yearly_metric_df['time_stamp'] = pd.to_datetime(yearly_metric_df['time_stamp'])
#         market_days_df = pd.DataFrame(df['time_stamp'])
#         for day in df:
#             if 
        


In [146]:
#keep track of which quarterly earnings release they are on
#when it passes, go to 0

def earnings_release(df, stock):
    quarterly_earnings_df = pd.read_csv(f'{path+stock}_EARNINGS.csv.gz')
    quarterly_earnings_df['time_stamp'] = pd.to_datetime(quarterly_earnings_df['time_stamp'])
    quarterly_earnings_df = quarterly_earnings_df.sort_values(by='time_stamp', ascending=True)
    print(df['time_stamp'])
    df["QUARTERLY_EARNINGS"] = np.nan
    i = 0
    for time in list(df['time_stamp']):
        if i > 23:
            i == 23
        x = int((quarterly_earnings_df['time_stamp'].iloc[i] - time).days)
        print("Real diff: ", x)
        print(quarterly_earnings_df['time_stamp'].iloc[i])
        print("dataframe date", time)
        if x == 0:
            i += 1
            #the next one find the difference between the time and the next quarterly start date
        if x > 5:
            x = 5
        if x < -4:
            x = -4
        df.loc[df['time_stamp'] == time, "QUARTERLY_EARNINGS"] = x
        print("i: ", i)
        print("x: ", x)
    return df

In [150]:
for stock in stocks:
    df = combine_datasets_by_stock(stock, daily_metrics)
    df = add_cal_dates(calendar_codes, df)
    print(len(df))
    # df = earnings_release(df, stock)


1126


In [157]:
# df = pd.read_csv("data_files/AAPL_DAILY_VOLATILITY.csv.gz")
import gzip
import csv
with gzip.open('data_files/AAPL_DAILY_VOLATILITY.csv.gz', 'rt', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['time_stamp', 'volatility']
['2020-01-02', '0.0004469713822737827']
['2020-01-03', '0.0005642232854149271']
['2020-01-06', '0.00043634451516300435']
['2020-01-07', '0.00045333456619145354']
['2020-01-08', '0.00048220200041154346']
['2020-01-09', '0.0005460024459991252']
['2020-01-10', '0.0005480989188031206']
['2020-01-13', '0.0003689431827329926']
['2020-01-14', '0.0007038323837815437']
['2020-01-15', '0.0005400891040806719']
['2020-01-16', '0.0003871022021347989']
['2020-01-17', '0.0003646512911999544']
['2020-01-21', '0.0003939418769406186']
['2020-01-22', '0.00037593472553700956']
['2020-01-23', '0.00042680758994495017']
['2020-01-24', '0.0005788110249672751']
['2020-01-27', '0.0007909569999800459']
['2020-01-28', '0.00047664810616956536']
['2020-01-29', '0.0007829268309170724']
['2020-01-30', '0.0006427346382720306']
['2020-01-31', '0.0009674761148564842']
['2020-02-03', '0.0008501780209467579']
['2020-02-04', '0.0005478272807642183']
['2020-02-05', '0.0006290849582680852']
['202

In [11]:
stock_dataframes = []
for metric in metrics:
    print(metric)
    df = pd.read_csv(f"data_files/AAPL_{metric}.csv.gz")
    stock_dataframes.append(df)
merged_df = reduce(lambda left, right: pd.merge(left, right, on='time_stamp', how='outer'), stock_dataframes)

moving_average
BALANCE_SHEET
CASH_FLOW
INCOME_STATEMENT
SHARES_OUTSTANDING


KeyError: 'time_stamp'

In [ ]:
stock_dataframes = []
for stock in stocks:
    for metric in metrics:
        df = pd.read_csv(f"{metric}_{stock}.csv")
        stock_dataframes.append(df)
    